# VGG
- Name: Minh Nguyen
- Date: 1/5/2025
- About: 
    - Known for the using 3x3 kernels throughout the networks
    - Deep architecture for learning hierarchical features
    - Consistent use of a small 3x3 kernels increases representational power without excessive parameters.
- Key Features:
    - Input Size: 224 x 224 x 3
    - Layers:
        - Stacks of small 3x3 convolutions with increasing depth.
    - Activation: ReLU activation for non-linearity.
    - Pooling: Employs max-pooling for downsampling.
    - FC: 2 FC layers with 4096 neurons
    - Regularization: Dropout in FC
    - Use Case: Simple dataset.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


class VGG(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        # Define the feature extraction layers
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 64, kernel_size=3, padding=1),  # 1 x 224 x 224 -> 64 x 224 x 224
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # 64 x 224 x 224 -> 64 x 224 x 224
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 64 x 224 x 224 -> 64 x 112 x 112
            
            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # 64 x 112 x 112 -> 128 x 112 x 112
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # 128 x 112 x 112 -> 128 x 112 x 112
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 128 x 112 x 112 -> 128 x 56 x 56
            
            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # 128 x 56 x 56 -> 256 x 56 x 56
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # 256 x 56 x 56 -> 256 x 56 x 56
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # 256 x 56 x 56 -> 256 x 56 x 56
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 256 x 56 x 56 -> 256 x 28 x 28
            
            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),  # 256 x 28 x 28 -> 512 x 28 x 28
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # 512 x 28 x 28 -> 512 x 28 x 28
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # 512 x 28 x 28 -> 512 x 28 x 28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 512 x 28 x 28 -> 512 x 14 x 14
            
            # Block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # 512 x 14 x 14 -> 512 x 14 x 14
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # 512 x 14 x 14 -> 512 x 14 x 14
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),  # 512 x 14 x 14 -> 512 x 14 x 14
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 512 x 14 x 14 -> 512 x 7 x 7
        )
        
        # Define the classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)  # Extract features
        x = x.view(x.size(0), -1)  # Flatten feature map
        x = self.classifier(x)  # Classify

class SmallVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(SmallVGG, self).__init__()
        # Define the feature extraction layers
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 8, kernel_size=3, padding=1),  # 1 x 28 x 28 -> 8 x 28 x 28
            nn.ReLU(inplace=True),
            nn.Conv2d(8, 8, kernel_size=3, padding=1),  # 8 x 28 x 28 -> 8 x 28 x 28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 8 x 28 x 28 -> 8 x 14 x 14
            
            # Block 2
            nn.Conv2d(8, 16, kernel_size=3, padding=1),  # 8 x 14 x 14 -> 16 x 14 x 14
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),  # 16 x 14 x 14 -> 16 x 14 x 14
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # 16 x 14 x 14 -> 16 x 7 x 7
        )
        
        # Define the classifier
        self.classifier = nn.Sequential(
            nn.Linear(16 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)  # Extract features
        x = x.view(x.size(0), -1)  # Flatten feature map
        x = self.classifier(x)  # Classify
        return x

# Prepare Fashion-MNIST dataset
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize to 224x224 for VGG
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training and testing settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = SmallVGG(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Testing Loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Using device: cpu
Epoch 1/10, Loss: 0.7721
Epoch 2/10, Loss: 0.3492
Epoch 3/10, Loss: 0.2886
Epoch 4/10, Loss: 0.2546
Epoch 5/10, Loss: 0.2312
Epoch 6/10, Loss: 0.2113
Epoch 7/10, Loss: 0.1977
Epoch 8/10, Loss: 0.1829
Epoch 9/10, Loss: 0.1674
Epoch 10/10, Loss: 0.1552
Test Accuracy: 91.60%
